## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN'  ], axis=1, inplace=True)


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [4]:
name_counts = application_df.NAME.value_counts()
name_counts

NAME
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: count, Length: 19568, dtype: int64

In [5]:
for name in name_counts[name_counts<6].index:
    application_df.NAME = application_df.NAME.str.replace(name, 'Other')

In [6]:
application_df.NAME.value_counts()

NAME
Other                                          19118
Other INC                                       1666
PARENT BOOSTER USA INC                          1260
TOPS CLUB INC                                    765
WASHINGTON STATE UNIVERSITY                      492
                                               ...  
TONASKET VISITORS & BUSINESS Other                 1
JONES RD COALITION FOR Other DRINKING WATER        1
OtherRAMENTO FIRE FIGHTERS PIPES AND DRUMS         1
MAMMOTH LAKES Other                                1
WATERHOUSE CHARITOther TR                          1
Name: count, Length: 1236, dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Get the application types to be replaced
application_types_to_replace = app_counts[app_counts < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
cv1 = classification_value_counts[classification_value_counts > 1]
cv1


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [11]:
# Calculate the number of data points for each unique value
classifications_to_replace = classification_value_counts[classification_value_counts < 1000].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Check to make sure replacement was successful
print(application_df['CLASSIFICATION'].value_counts())


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df,dtype='int')
dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_A Other,NAME_A Other PLACE COMMUNITY DEVELOPMENT CORPORATION,NAME_AACE INTERNATIONAL,NAME_ACADEMICS PLUS CHARTER SCHOOL Other INC,NAME_ACADEMY OF Other,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACONA Other,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [13]:
# Split our preprocessed data into our features and target arrays
y = dummies_df.IS_SUCCESSFUL
X = dummies_df.drop('IS_SUCCESSFUL',axis=1)

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
# Define the number of features
n_features = X_train_scaled.shape[1]  


## Compile, Train and Evaluate the Model

In [17]:
# Define the model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=n_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=15, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │       102,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │           465 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 105,311 (411.37 KB)

 Trainable params: 105,311 (411.37 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [19]:
len(X_train_scaled)

25724

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, batch_size=1, epochs=100)


Epoch 1/100
25724/25724 ━━━━━━━━━━━━━━━━━━━━ 13s 488us/step - accuracy: 0.7507 - loss: 0.5097
Epoch 2/100
25724/25724 ━━━━━━━━━━━━━━━━━━━━ 12s 483us/step - accuracy: 0.7881 - loss: 0.4524
Epoch 3/100
25724/25724 ━━━━━━━━━━━━━━━━━━━━ 12s 480us/step - accuracy: 0.7930 - loss: 0.4389
Epoch 4/100
25724/25724 ━━━━━━━━━━━━━━━━━━━━ 12s 485us/step - accuracy: 0.7966 - loss: 0.4351
Epoch 5/100
25724/25724 ━━━━━━━━━━━━━━━━━━━━ 13s 489us/step - accuracy: 0.7984 - loss: 0.4251
Epoch 6/100
25724/25724 ━━━━━━━━━━━━━━━━━━━━ 12s 482us/step - accuracy: 0.8006 - loss: 0.4284
Epoch 7/100
25724/25724 ━━━━━━━━━━━━━━━━━━━━ 12s 482us/step - accuracy: 0.7963 - loss: 0.4328
Epoch 8/100
25724/25724 ━━━━━━━━━━━━━━━━━━━━ 13s 489us/step - accuracy: 0.7973 - loss: 0.4293
Epoch 9/100
25724/25724 ━━━━━━━━━━━━━━━━━━━━ 12s 484us/step - accuracy: 0.8067 - loss: 0.4226
Epoch 10/100
25724/25724 ━━━━━━━━━━━━━━━━━━━━ 12s 485us/step - accuracy: 0.8060 - loss: 0.4212
Epoch 11/100
25724/25724 ━━━━━━━━━━━━━━━━━━━━ 13s 492us/ste

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 420us/step - accuracy: 0.7871 - loss: 0.4667
Loss: 0.4667433500289917, Accuracy: 0.7870553731918335


In [21]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")
